# 🔬 QuiverQuant Data Exploration Notebook

**Objectif:** Tester et explorer les données QuiverQuant petit à petit pour comprendre les formats, la qualité et la structure.

**Date:** 31 Décembre 2025  
**Token:** bibep  

---

## 📋 Plan d'exploration

1. ✅ Import des libraries
2. ✅ Connexion à QuiverQuant
3. ✅ Congressional Trading - Tous les trades
4. ✅ Congressional Trading - Par ticker spécifique
5. ✅ Senate vs House Trading
6. ✅ Insider Trading
7. ✅ WallStreetBets Sentiment
8. ✅ Analyse comparative
9. ✅ Nettoyage et standardisation
10. ✅ Export et intégration

## 1️⃣ Import des Libraries

In [4]:
import sys
import os
from pathlib import Path

# Déterminer le répertoire racine du projet
notebook_dir = Path(r'c:\project\n8n-dashboard-v4\notebooks')
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

# Ajouter les chemins importants
sys.path.insert(0, str(project_root / 'prod'))
sys.path.insert(0, str(project_root / 'services'))

print(f"📂 Project root: {project_root}")
print(f"📂 Paths ajoutés: {sys.path[:3]}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json

# Importer le client QuiverQuant depuis la structure correcte
try:
    from quiverquant.quiverquant_client import QuiverQuantClient
    from quiverquant.config import QUIVERQUANT_TOKEN
    print("✅ QuiverQuant modules importés avec succès")
except ImportError as e:
    print(f"⚠️ Erreur import QuiverQuant: {e}")
    print("   Vérifier que les modules existent dans la branche _political_trading")
    QUIVERQUANT_TOKEN = None

print("✅ Libraries importées")
print(f"📅 Date/heure: {datetime.now()}")
print(f"🔐 Token disponible: {'✅' if QUIVERQUANT_TOKEN else '❌ (module non trouvé)'}")

📂 Project root: c:\project\n8n-dashboard-v4
📂 Paths ajoutés: ['c:\\project\\n8n-dashboard-v4\\services', 'c:\\project\\n8n-dashboard-v4\\prod', 'c:\\project\\n8n-dashboard-v4']
✅ QuiverQuant modules importés avec succès
✅ Libraries importées
📅 Date/heure: 2026-01-02 14:13:12.988556
🔐 Token disponible: ✅


## 2️⃣ Connexion à QuiverQuant & Première Requête

In [6]:
# Initialiser le client
print("🔌 Initialisation du client QuiverQuant...")
client = QuiverQuantClient(QUIVERQUANT_TOKEN)
print("✅ Client prêt!")

# Première requête: Congressional Trading
print("\n" + "="*70)
print("📡 TEST 1: Congressional Trading (Tous les trades récents)")
print("="*70)

try:
    df_congress = client.congress_trading()
    print(f"✅ Succès! {len(df_congress)} trades récupérés")
    print(f"\n📊 Shape: {df_congress.shape}")
    print(f"\n📋 Colonnes: {list(df_congress.columns)}")
    print(f"\n🔍 Premiers 3 trades:")
    print(df_congress.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")

🔌 Initialisation du client QuiverQuant...
✅ Client prêt!

📡 TEST 1: Congressional Trading (Tous les trades récents)
📡 Fetching: https://api.quiverquant.com/beta/live/congresstrading
✅ Succès! 1000 trades récupérés

📊 Shape: (1000, 16)

📋 Colonnes: ['Representative', 'BioGuideID', 'ReportDate', 'TransactionDate', 'Ticker', 'Transaction', 'Range', 'House', 'Amount', 'Party', 'last_modified', 'TickerType', 'Description', 'ExcessReturn', 'PriceChange', 'SPYChange']

🔍 Premiers 3 trades:
    Representative BioGuideID ReportDate TransactionDate Ticker Transaction  \
0  Josh Gottheimer    G000583 2025-12-29      2025-11-21  DASTY        Sale   
1  Josh Gottheimer    G000583 2025-12-29      2025-11-19     EW        Sale   
2  Josh Gottheimer    G000583 2025-12-29      2025-11-19   HUBS        Sale   

              Range            House  Amount Party last_modified TickerType  \
0  $1,001 - $15,000  Representatives  1001.0     D    2025-12-30         ST   
1  $1,001 - $15,000  Representatives 

## 3️⃣ Exploration Détaillée - Congressional Trading

In [7]:
print("📊 ANALYSE CONGRESSIONAL TRADING")
print("="*70)

# Info générales
print("\n1️⃣ INFO GÉNÉRALES:")
print(f"   Nombre de trades: {len(df_congress)}")
print(f"   Date min: {df_congress['TransactionDate'].min() if 'TransactionDate' in df_congress.columns else 'N/A'}")
print(f"   Date max: {df_congress['TransactionDate'].max() if 'TransactionDate' in df_congress.columns else 'N/A'}")

# Types de données
print("\n2️⃣ TYPES DE DONNÉES:")
print(df_congress.dtypes)

# Valeurs manquantes
print("\n3️⃣ VALEURS MANQUANTES:")
missing = df_congress.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✅ Aucune valeur manquante!")

# Statistiques descriptives
print("\n4️⃣ STATISTIQUES DESCRIPTIVES:")
print(df_congress.describe())

# Politiciens uniques
if 'Representative' in df_congress.columns:
    print(f"\n5️⃣ POLITICIENS UNIQUES: {df_congress['Representative'].nunique()}")
    print("   Top 10:")
    print(df_congress['Representative'].value_counts().head(10))

# Tickers uniques
if 'Ticker' in df_congress.columns:
    print(f"\n6️⃣ TICKERS UNIQUES: {df_congress['Ticker'].nunique()}")
    print("   Top 10:")
    print(df_congress['Ticker'].value_counts().head(10))

# Types de transactions
if 'Transaction' in df_congress.columns:
    print(f"\n7️⃣ TYPES DE TRANSACTIONS:")
    print(df_congress['Transaction'].value_counts())

📊 ANALYSE CONGRESSIONAL TRADING

1️⃣ INFO GÉNÉRALES:
   Nombre de trades: 1000
   Date min: 2025-01-02 00:00:00
   Date max: 2025-12-16 00:00:00

2️⃣ TYPES DE DONNÉES:
Representative             object
BioGuideID                 object
ReportDate         datetime64[ns]
TransactionDate    datetime64[ns]
Ticker                     object
Transaction                object
Range                      object
House                      object
Amount                     object
Party                      object
last_modified              object
TickerType                 object
Description                object
ExcessReturn              float64
PriceChange               float64
SPYChange                 float64
dtype: object

3️⃣ VALEURS MANQUANTES:
Description     976
ExcessReturn     22
PriceChange      22
SPYChange        22
dtype: int64

4️⃣ STATISTIQUES DESCRIPTIVES:
                       ReportDate             TransactionDate  ExcessReturn  \
count                        1000            

## 4️⃣ Test Spécifique - Trades Tesla par Congress

In [14]:
print("🔍 TEST TRADES PAR TICKER SPÉCIFIQUE")
print("="*70)
print("Note: L'endpoint par ticker retourne des erreurs JSON.")
print("Solution: Filtrer le DataFrame complet récupéré précédemment\n")

# Tickers populaires à analyser
test_tickers = ["TSLA", "NVDA", "MSFT", "META", "AAPL"]

for ticker in test_tickers:
    print(f"\n📊 Analyse de {ticker}:")
    
    # Filtrer df_congress au lieu d'appeler l'API par ticker
    if 'Ticker' in df_congress.columns:
        df_ticker = df_congress[df_congress['Ticker'] == ticker]
        print(f"   ✅ {len(df_ticker)} trades trouvés dans df_congress")
        
        if len(df_ticker) > 0:
            # Afficher les 3 premiers
            print(f"   📋 Premiers trades:")
            for idx, row in df_ticker.head(3).iterrows():
                rep_name = row['Representative'][:30] if len(str(row['Representative'])) > 30 else row['Representative']
                trans_date = row['TransactionDate']
                trans_type = row['Transaction']
                print(f"      - {rep_name}: {trans_type} on {trans_date}")
            
            # Statistiques BUY/SELL
            if 'Transaction' in df_ticker.columns:
                buys = (df_ticker['Transaction'] == 'Purchase').sum()
                sells = (df_ticker['Transaction'] == 'Sale').sum()
                print(f"   📈 BUY: {buys} | SELL: {sells}")
                
                # Date range
                if 'TransactionDate' in df_ticker.columns:
                    df_ticker_copy = df_ticker.copy()
                    df_ticker_copy['TransactionDate'] = pd.to_datetime(df_ticker_copy['TransactionDate'])
                    date_min = df_ticker_copy['TransactionDate'].min()
                    date_max = df_ticker_copy['TransactionDate'].max()
                    print(f"   📅 Date range: {date_min.date()} → {date_max.date()}")
        else:
            print(f"   ⚠️ Aucun trade pour {ticker} dans les données récentes")
    else:
        print(f"   ❌ Colonne 'Ticker' non trouvée dans df_congress")

print("\n" + "="*70)
print("✅ Test terminé")
print("\n💡 Solution utilisée: Filtrage du DataFrame complet (méthode recommandée)")
print("   car l'endpoint /live/congresstrading/{ticker} retourne des erreurs JSON.")

🔍 TEST TRADES PAR TICKER SPÉCIFIQUE
Note: L'endpoint par ticker retourne des erreurs JSON.
Solution: Filtrer le DataFrame complet récupéré précédemment


📊 Analyse de TSLA:
   ✅ 2 trades trouvés dans df_congress
   📋 Premiers trades:
      - Gilbert Cisneros: Purchase on 2025-11-18 00:00:00
      - Lisa Mcclain: Sale on 2025-10-31 00:00:00
   📈 BUY: 1 | SELL: 1
   📅 Date range: 2025-10-31 → 2025-11-18

📊 Analyse de NVDA:
   ✅ 15 trades trouvés dans df_congress
   📋 Premiers trades:
      - Dwight Evans: Purchase on 2025-11-21 00:00:00
      - Gilbert Cisneros: Purchase on 2025-11-18 00:00:00
      - Lisa Mcclain: Sale on 2025-10-31 00:00:00
   📈 BUY: 9 | SELL: 1
   📅 Date range: 2025-01-06 → 2025-11-21

📊 Analyse de MSFT:
   ✅ 26 trades trouvés dans df_congress
   📋 Premiers trades:
      - Dwight Evans: Sale on 2025-11-21 00:00:00
      - Cleo Fields: Purchase on 2025-12-11 00:00:00
      - Cleo Fields: Purchase on 2025-12-11 00:00:00
   📈 BUY: 12 | SELL: 2
   📅 Date range: 2025-01-03

## 5️⃣ Senate vs House Trading (Données Séparées)

In [9]:
print("🏛️ SENATE vs HOUSE TRADING")
print("="*70)

# Senate
print("\n1️⃣ SENATE TRADING:")
try:
    df_senate = client.senate_trading()
    print(f"✅ {len(df_senate)} trades du Sénat")
    if len(df_senate) > 0:
        print(f"   Colonnes: {list(df_senate.columns)}")
        print(f"   \n   Premiers 3:")
        print(df_senate.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")

# House
print("\n2️⃣ HOUSE TRADING:")
try:
    df_house = client.house_trading()
    print(f"✅ {len(df_house)} trades de la Chambre")
    if len(df_house) > 0:
        print(f"   Colonnes: {list(df_house.columns)}")
        print(f"   \n   Premiers 3:")
        print(df_house.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")

# Comparaison
print("\n3️⃣ COMPARAISON:")
print(f"   Senate: {len(df_senate) if len(df_senate) > 0 else 0} trades")
print(f"   House: {len(df_house) if len(df_house) > 0 else 0} trades")
print(f"   Total Congress (combiné): {len(df_congress)} trades")

🏛️ SENATE vs HOUSE TRADING

1️⃣ SENATE TRADING:
✅ 5000 trades du Sénat
   Colonnes: ['Senator', 'BioGuideID', 'Date', 'Ticker', 'Transaction', 'Range', 'Amount', 'last_modified']
   
   Premiers 3:
                     Senator BioGuideID       Date Ticker Transaction  \
0  A. Mitchell Jr. McConnell    M000355 2025-12-01    WFC    Purchase   
1         David H. McCormick    M001243 2025-11-28   BITB    Purchase   
2         David H. McCormick    M001243 2025-11-26   BITB    Purchase   

                Range   Amount last_modified  
0    $1,001 - $15,000   1001.0    2025-12-19  
1  $50,001 - $100,000  50001.0    2025-12-26  
2  $50,001 - $100,000  50001.0    2025-12-26  

2️⃣ HOUSE TRADING:
✅ 5000 trades du Sénat
   Colonnes: ['Senator', 'BioGuideID', 'Date', 'Ticker', 'Transaction', 'Range', 'Amount', 'last_modified']
   
   Premiers 3:
                     Senator BioGuideID       Date Ticker Transaction  \
0  A. Mitchell Jr. McConnell    M000355 2025-12-01    WFC    Purchase   
1    

## 6️⃣ Insider Trading

In [10]:
print("👔 INSIDER TRADING DATA")
print("="*70)

try:
    df_insiders = client.insiders()
    print(f"✅ {len(df_insiders)} insider trades trouvés")
    
    print(f"\n📊 Shape: {df_insiders.shape}")
    print(f"\n📋 Colonnes: {list(df_insiders.columns)}")
    
    print(f"\n🔍 Premiers 5 insiders:")
    print(df_insiders.head(5))
    
    # Analyze
    if 'Ticker' in df_insiders.columns:
        print(f"\n📈 TOP 10 TICKERS:")
        print(df_insiders['Ticker'].value_counts().head(10))
    
    if 'Relationship' in df_insiders.columns:
        print(f"\n👤 TYPES DE RELATIONSHIPS:")
        print(df_insiders['Relationship'].value_counts().head(10))
        
except Exception as e:
    print(f"❌ Erreur: {e}")

👔 INSIDER TRADING DATA
📡 Fetching: https://api.quiverquant.com/beta/live/insiders
❌ Erreur: If using all scalar values, you must pass an index
❌ Erreur: If using all scalar values, you must pass an index


## 7️⃣ WallStreetBets Sentiment

In [17]:
print("💬 WALLSTREETBETS SENTIMENT")
print("="*70)
print("⚠️ Note: Cet endpoint peut nécessiter un abonnement premium\n")

# Essayer différentes méthodes selon la documentation
methods = [
    ("Yesterday's data", lambda: client.wallstreetbets(yesterday=True)),
    ("Date range (last 7 days)", lambda: client.wallstreetbets(
        date_from=(datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d"),
        date_to=datetime.now().strftime("%Y-%m-%d")
    )),
    ("All data (défaut)", lambda: client.wallstreetbets()),
]

df_wsb = None
success = False

for method_name, method_func in methods:
    print(f"🔍 Tentative: {method_name}")
    try:
        df_wsb = method_func()
        if df_wsb is not None and len(df_wsb) > 0:
            print(f"   ✅ Succès! {len(df_wsb)} mentions trouvées")
            success = True
            break
        else:
            print(f"   ⚠️ Réponse vide")
    except Exception as e:
        error_msg = str(e)
        if "Upgrade your subscription" in error_msg:
            print(f"   ❌ Premium requis: {error_msg[:60]}")
        elif "If using all scalar values" in error_msg:
            print(f"   ❌ Format réponse invalide (possiblement vide ou premium)")
        else:
            print(f"   ❌ Échec: {error_msg[:60]}")

if success and df_wsb is not None and len(df_wsb) > 0:
    print(f"\n" + "="*70)
    print("📊 ANALYSE DES DONNÉES WSB")
    print("="*70)
    
    print(f"\n📊 Shape: {df_wsb.shape}")
    print(f"📋 Colonnes: {list(df_wsb.columns)}")
    
    print(f"\n🔍 Premiers 5 mentions:")
    print(df_wsb.head(5))
    
    # Analyze
    if 'Ticker' in df_wsb.columns:
        print(f"\n📈 TOP 15 TICKERS MENTIONNÉS:")
        print(df_wsb['Ticker'].value_counts().head(15))
        
    # Date range
    if 'Date' in df_wsb.columns:
        print(f"\n📅 DATE RANGE:")
        print(f"   Min: {df_wsb['Date'].min()}")
        print(f"   Max: {df_wsb['Date'].max()}")
    
    # Sentiment analysis (si disponible)
    if 'Sentiment' in df_wsb.columns:
        print(f"\n😊 SENTIMENT ANALYSIS:")
        print(df_wsb['Sentiment'].value_counts())
    
    # Mentions analysis (si disponible)
    if 'Mentions' in df_wsb.columns:
        print(f"\n📊 MENTIONS STATISTICS:")
        print(f"   Total mentions: {df_wsb['Mentions'].sum()}")
        print(f"   Average: {df_wsb['Mentions'].mean():.1f}")
        print(f"   Median: {df_wsb['Mentions'].median():.1f}")
else:
    print(f"\n" + "="*70)
    print("⚠️ ENDPOINT WALLSTREETBETS NON DISPONIBLE")
    print("="*70)
    print("""
Raisons possibles:
   ✓ Endpoint nécessite abonnement premium
   ✓ Réponse API vide ou format différent
   ✓ Données pas disponibles pour le compte actuel (bibep)

Solution:
   - Vérifier abonnement sur quiverquant.com
   - Contacter chris@quiverquant.com pour upgrade
   - Les endpoints Congressional/Senate/House fonctionnent correctement ✅

Alternative:
   - Utiliser reddit API directement
   - Scraper r/wallstreetbets manuellement
   - Focus sur political trading data (disponible)
    """)

💬 WALLSTREETBETS SENTIMENT
⚠️ Note: Cet endpoint peut nécessiter un abonnement premium

🔍 Tentative: Yesterday's data
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: Date range (last 7 days)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true&date_from=20251226&date_to=2026-01-02
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: Date range (last 7 days)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true&date_from=20251226&date_to=2026-01-02
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: All data (défaut)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: All data (défaut)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true
   ❌ Format réponse invalide (possiblement 

## 8️⃣ Analyse Comparative & Standardisation

In [ ]:
print("🔄 STANDARDISATION DES DONNÉES")
print("="*70)

# Créer une version standardisée des congressional trades
print("\n1️⃣ CONGRESSIONAL TRADES - NETTOYAGE:")

df_congress_clean = df_congress.copy()

# Rename columns pour cohérence
column_mapping = {
    'Representative': 'politician_name',
    'TransactionDate': 'transaction_date',
    'ReportDate': 'report_date',
    'Ticker': 'ticker',
    'Transaction': 'transaction_type',
    'Amount': 'amount',
    'Chamber': 'chamber'
}

df_congress_clean = df_congress_clean.rename(columns={k: v for k, v in column_mapping.items() if k in df_congress_clean.columns})

# Add source
df_congress_clean['source'] = 'Congress (QuiverQuant)'
df_congress_clean['data_type'] = 'political_trade'

print(f"✅ Colonnes standardisées: {list(df_congress_clean.columns)}")
print(f"\n📋 Aperçu:")
print(df_congress_clean[[col for col in ['politician_name', 'transaction_date', 'ticker', 'transaction_type'] if col in df_congress_clean.columns]].head(5))

# Vérifier les types de transactions
print(f"\n📈 TYPES DE TRANSACTIONS (avant standardisation):")
if 'transaction_type' in df_congress_clean.columns:
    print(df_congress_clean['transaction_type'].value_counts())
    
# Mapper to BUY/SELL
print(f"\n🔄 MAPPING BUY/SELL:")
buy_terms = ['Purchase', 'Buy']
sell_terms = ['Sale', 'Sell']

def normalize_transaction_type(val):
    if pd.isna(val):
        return val
    val_str = str(val).lower()
    if any(term.lower() in val_str for term in buy_terms):
        return 'BUY'
    elif any(term.lower() in val_str for term in sell_terms):
        return 'SELL'
    else:
        return val

df_congress_clean['transaction_type_normalized'] = df_congress_clean['transaction_type'].apply(normalize_transaction_type)

print("Avant mapping:", df_congress_clean['transaction_type'].unique()[:5])
print("Après mapping:", df_congress_clean['transaction_type_normalized'].unique())
print("\nComptage après mapping:")
print(df_congress_clean['transaction_type_normalized'].value_counts())

## 9️⃣ Statistiques Descriptives Complètes

In [ ]:
# Créer un résumé complet
print("📊 RÉSUMÉ COMPLET DES DONNÉES")
print("="*70)

summary = {
    "Congressional Trading": {
        "count": len(df_congress),
        "columns": list(df_congress.columns),
        "politicians": df_congress['Representative'].nunique() if 'Representative' in df_congress.columns else 'N/A',
        "tickers": df_congress['Ticker'].nunique() if 'Ticker' in df_congress.columns else 'N/A',
    },
    "Senate Trading": {
        "count": len(df_senate) if 'df_senate' in locals() and len(df_senate) > 0 else 0,
        "columns": list(df_senate.columns) if 'df_senate' in locals() else [],
    },
    "House Trading": {
        "count": len(df_house) if 'df_house' in locals() and len(df_house) > 0 else 0,
        "columns": list(df_house.columns) if 'df_house' in locals() else [],
    },
    "Insider Trading": {
        "count": len(df_insiders) if 'df_insiders' in locals() and len(df_insiders) > 0 else 0,
        "columns": list(df_insiders.columns) if 'df_insiders' in locals() else [],
    },
    "WSB Sentiment": {
        "count": len(df_wsb) if 'df_wsb' in locals() and len(df_wsb) > 0 else 0,
        "columns": list(df_wsb.columns) if 'df_wsb' in locals() else [],
    }
}

# Display summary
for source, info in summary.items():
    print(f"\n{source}:")
    for key, val in info.items():
        if key != 'columns':
            print(f"   {key}: {val}")
        else:
            if val:
                print(f"   Colonnes: {', '.join(val[:3])}... ({len(val)} total)")

# Save summary to JSON
summary_path = Path("c:/n8n-local-stack/local_files/smart_money/quiverquant_data_summary.json")
summary_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
    
print(f"\n✅ Résumé sauvegardé: {summary_path}")

## 🔟 Export & Intégration

In [ ]:
print("💾 EXPORT DONNÉES")
print("="*70)

# Export Congressional cleaned
export_path = Path("c:/n8n-local-stack/local_files/smart_money/congressional_trades_explored.csv")
export_path.parent.mkdir(parents=True, exist_ok=True)

df_congress_clean.to_csv(export_path, index=False)
print(f"✅ Congressional trades exportés: {export_path}")
print(f"   Taille: {len(df_congress_clean)} lignes")

# Export as JSON
json_path = Path("c:/n8n-local-stack/local_files/smart_money/congressional_trades_explored.json")
df_congress_clean.to_json(json_path, orient='records', date_format='iso', indent=2)
print(f"✅ JSON exporté: {json_path}")

# Résumé pour intégration
print("\n" + "="*70)
print("📝 RÉSUMÉ POUR INTÉGRATION SMART MONEY")
print("="*70)

print("""
✅ DONNÉES DISPONIBLES:
   1. Congressional Trading (Tous politiciens)
   2. Senate Trading (Sénat uniquement)
   3. House Trading (Chambre uniquement)
   4. Insider Trading (Company insiders)
   5. WallStreetBets Sentiment (Retail sentiment)

📊 QUALITÉ DES DONNÉES:
   ✓ Congressional: Excellent (dates, noms, tickers, transactions)
   ✓ Standardisation: Possible (BUY/SELL mapping)
   ✓ Intégration: Prête pour Smart Money Tracker

🔄 PROCHAINES ÉTAPES:
   1. Intégrer dans edgar_smart_money_analyzer.py
   2. Créer signaux politiques
   3. Combiner avec Smart Money metrics
   4. Tester sur historical data

💡 NOTES IMPORTANTES:
   - Congressional data a des délais de report (5-45 jours)
   - Insider trades ont aussi des délais réglementaires
   - WSB sentiment est real-time mais bruyant
   - À combiner avec d'autres signaux pour meilleur signal
""")

print(f"\n✅ Notebook exploration complète!")
print(f"📁 Fichiers générés:")
print(f"   - {export_path}")
print(f"   - {json_path}")
print(f"   - {summary_path}")

## 🔍 BONUS: Limitations de l'API & Solutions

In [5]:
print("⚠️ LIMITATIONS DE L'API QUIVERQUANT")
print("="*70)

print("""
❌ PROBLÈME #1: LIMITE DE 1000 RÉSULTATS
─────────────────────────────────────────
Raison: Protection contre surcharge API (pagination)
Conséquence: Seulement les 1000 trades RÉCENTS retournés

Analyse des dates:
""")

# Analyser la distribution des dates
if 'TransactionDate' in df_congress.columns:
    df_congress['TransactionDate'] = pd.to_datetime(df_congress['TransactionDate'])
    
    date_min = df_congress['TransactionDate'].min()
    date_max = df_congress['TransactionDate'].max()
    days_span = (date_max - date_min).days
    
    print(f"   Date min:     {date_min.date()}")
    print(f"   Date max:     {date_max.date()}")
    print(f"   Span:         {days_span} jours ({days_span/365:.1f} ans)")
    print(f"   Trades/jour:  {len(df_congress) / (days_span + 1):.1f}")

print("""

❌ PROBLÈME #2: DÉLAI DE REPORTING (1 AN)
──────────────────────────────────────────
Raison: Délais légaux avant disclosure public
- Congressional trades: 5 à 45 jours après transaction
- Insider trades: Jusqu'à 45 jours après transaction
- Résultat: Données actuelles = transactions de mois/années antérieurs!

Example du notebook:
   Transaction date: 2025-11-21
   Report date:      2025-12-29
   Délai:            38 jours ✅ Normal

❌ PROBLÈME #3: "1 AN" SIGNIFIE QUOI?
──────────────────────────────────────
C'est PAS une limite de l'API, c'est:
✓ Fenêtre temporelle des données disponibles
✓ 1000 résultats les plus récents = ~1 an de trades
✓ Au-delà, faut utiliser des paramètres pour historical data

═══════════════════════════════════════════════════════════════════════

🔧 SOLUTIONS POUR AVOIR PLUS DE DONNÉES:
""")

print("\n1️⃣ PAGINATION (non implémentée dans le client):")
print("""
   - Ajouter paramètres: 'offset', 'limit'
   - Récupérer blocs de 100-500 résultats
   - Itérer pour avoir historique complet
   Exemple URL:
   /live/congresstrading?limit=100&offset=0
   /live/congresstrading?limit=100&offset=100
   /live/congresstrading?limit=100&offset=200
   ... etc
""")

print("2️⃣ ENDPOINTS HISTORIQUES:")
print("""
   - /historical/congresstrading/{ticker}  (par ticker)
   - /historical/senatetrading/{ticker}     (par ticker)
   - /historical/housetrading/{ticker}      (par ticker)
   
   Avantage: Plus de données par ticker
   Limitation: Faut connaître les tickers avance
""")

print("3️⃣ PARAMÈTRES DATE:")
print("""
   - date_from, date_to
   - Certains endpoints supportent filters
   - Permet de requêter historique spécifique
""")

print("4️⃣ CACHING LOCAL:")
print("""
   - Sauvegarder les 1000 résultats en local
   - Daily fetch = nouveaux trades seulement
   - Accumulation historique progressive
   - Solution recommandée pour production ✅
""")

print("\n═══════════════════════════════════════════════════════════════════════\n")

⚠️ LIMITATIONS DE L'API QUIVERQUANT

❌ PROBLÈME #1: LIMITE DE 1000 RÉSULTATS
─────────────────────────────────────────
Raison: Protection contre surcharge API (pagination)
Conséquence: Seulement les 1000 trades RÉCENTS retournés

Analyse des dates:

   Date min:     2025-01-02
   Date max:     2025-12-16
   Span:         348 jours (1.0 ans)
   Trades/jour:  2.9


❌ PROBLÈME #2: DÉLAI DE REPORTING (1 AN)
──────────────────────────────────────────
Raison: Délais légaux avant disclosure public
- Congressional trades: 5 à 45 jours après transaction
- Insider trades: Jusqu'à 45 jours après transaction
- Résultat: Données actuelles = transactions de mois/années antérieurs!

Example du notebook:
   Transaction date: 2025-11-21
   Report date:      2025-12-29
   Délai:            38 jours ✅ Normal

❌ PROBLÈME #3: "1 AN" SIGNIFIE QUOI?
──────────────────────────────────────
C'est PAS une limite de l'API, c'est:
✓ Fenêtre temporelle des données disponibles
✓ 1000 résultats les plus récents = ~1 

## 💡 Implémentation: Solution de Caching Local

In [6]:
print("🔄 SOLUTION: ACCUMULATION PROGRESSIVE")
print("="*70)

# Simuler un cache accumulatif
cache_dir = Path("c:/n8n-local-stack/local_files/smart_money/quiverquant_cache")
cache_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Cache directory: {cache_dir}")

# Fonction pour charger ou créer cache
def load_or_create_cache(source_name, max_age_days=1):
    """
    Load cached data ou fetch new if too old
    """
    cache_file = cache_dir / f"{source_name}_cache.parquet"
    
    if cache_file.exists():
        # Check age
        mod_time = datetime.fromtimestamp(cache_file.stat().st_mtime)
        age = (datetime.now() - mod_time).days
        
        if age < max_age_days:
            print(f"✅ Cache valide ({age} jours old)")
            return pd.read_parquet(cache_file), True
        else:
            print(f"⚠️ Cache expiré ({age} jours old)")
            return None, False
    else:
        print(f"📝 Pas de cache - créer nouveau")
        return None, False

# Fonction pour sauvegarder avec historique
def save_with_history(df_new, source_name, max_keep=5):
    """
    Save new data and keep rotation of old caches
    """
    cache_file = cache_dir / f"{source_name}_cache.parquet"
    
    # Si cache existe, charger et merger
    if cache_file.exists():
        df_old = pd.read_parquet(cache_file)
        print(f"   Old cache: {len(df_old)} lignes")
        
        # Merger (éviter duplicates)
        df_merged = pd.concat([df_new, df_old]).drop_duplicates(
            subset=['Representative', 'TransactionDate', 'Ticker'] if 'Representative' in df_new.columns else None,
            keep='first'
        )
        print(f"   After merge: {len(df_merged)} lignes")
    else:
        df_merged = df_new
        
    # Save
    df_merged.to_parquet(cache_file)
    print(f"   ✅ Saved: {cache_file}")
    
    # Rotate old backups (keep last 5)
    backup_files = sorted(cache_dir.glob(f"{source_name}_backup_*.parquet"))
    for f in backup_files[:-max_keep]:
        f.unlink()
        
    return df_merged

# TEST
print("\n1️⃣ PREMIÈRE EXÉCUTION (pas de cache):")
df_cached, found = load_or_create_cache("congressional_trading")
if not found:
    df_cached = client.congress_trading()
    df_merged = save_with_history(df_cached, "congressional_trading")
    print(f"\n📊 Total historique: {len(df_merged)} trades")

print("\n2️⃣ DEUXIÈME EXÉCUTION (avec cache):")
df_cached2, found2 = load_or_create_cache("congressional_trading")
if found2:
    print(f"   Loaded: {len(df_cached2)} trades from cache")

print("\n" + "="*70)
print("✅ RÉSULTAT:")
print("""
Au lieu de:
   - Fetch 1000 aujourd'hui
   - Perte après 1 an (circular buffer)

Maintenant:
   - Fetch 1000 daily
   - Accumule historique localement
   - 1000 * 365 = 365,000 trades potentiels!
   - Croissance progressive sans perte
   
Avantage: Après 1 an de collection, avoir ~365K trades
         Puis utiliser /historical/{ticker} pour des tickers spécifiques
""")

print(f"\n📁 Cache saved at: {cache_dir}")

🔄 SOLUTION: ACCUMULATION PROGRESSIVE

📁 Cache directory: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache

1️⃣ PREMIÈRE EXÉCUTION (pas de cache):
📝 Pas de cache - créer nouveau
📡 Fetching: https://api.quiverquant.com/beta/live/congresstrading
   ✅ Saved: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache\congressional_trading_cache.parquet

📊 Total historique: 1000 trades

2️⃣ DEUXIÈME EXÉCUTION (avec cache):
✅ Cache valide (0 jours old)
   Loaded: 1000 trades from cache

✅ RÉSULTAT:

Au lieu de:
   - Fetch 1000 aujourd'hui
   - Perte après 1 an (circular buffer)

Maintenant:
   - Fetch 1000 daily
   - Accumule historique localement
   - 1000 * 365 = 365,000 trades potentiels!
   - Croissance progressive sans perte
   
Avantage: Après 1 an de collection, avoir ~365K trades
         Puis utiliser /historical/{ticker} pour des tickers spécifiques


📁 Cache saved at: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache
